In [1]:
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=64,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)


In [3]:
USE_GPU = torch.cuda.is_available()

In [4]:
from allennlp.common.checks import ConfigurationError

In [5]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

## Prepare dataset

In [6]:
label_cols = ["science", "funny"]

# label_cols = ['science', 'funny', 'engineering', 'compsci',
#                 'machinelearning', 'datascience', 'math', 'statistics']

In [7]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

class RedditDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token],
                         labels: np.ndarray=None,
                        id: str=None,) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        fields["label"] = ArrayField(array=labels)
    
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        # Reads in the pickle file and inputs the document, label, and id to make an instance
        df = pd.read_pickle(file_path)
        #df = df[df.labels<2]
        # Imports data into the tokenizer
        for index, row in df.iterrows():
            #print(row.documents)
            yield self.text_to_instance([Token(x) for x in self.tokenizer(row.documents)],
                                        row[label_cols].values,
                                        index)
            
            

## Prepare token handlers

In [8]:
from allennlp.data.token_indexers import PretrainedBertIndexer

token_indexer = PretrainedBertIndexer(
    pretrained_model="bert-base-uncased",
    max_pieces=config.max_seq_len,
    do_lowercase=True,
 )
# apparently we need to truncate the sequence here, which is a stupid design decision
def tokenizer(s: str):
    return token_indexer.wordpiece_tokenizer(s)[:config.max_seq_len - 2]

In [9]:
reader = RedditDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [10]:
#train_ds, test_ds = (reader.read(fname) for fname in ['reddit_thread_label_small','reddit_thread_label_small'])
train_ds = reader.read('reddit_thread_label_small_train')

2000it [00:01, 1165.17it/s]


In [11]:
len(train_ds)

2000

In [12]:
vars(train_ds[0].fields["tokens"])

{'tokens': [[UNK],
  are,
  using,
  imaging,
  tests,
  to,
  show,
  for,
  the,
  first,
  time,
  that,
  fr,
  ##uc,
  ##tose,
  can,
  trigger,
  brain,
  changes,
  that,
  may,
  lead,
  to,
  over,
  ##ea,
  ##ting,
  ##.,
  [UNK],
  is,
  a,
  sugar,
  that,
  sat,
  ##ura,
  ##tes,
  the,
  [UNK],
  diet,
  ##.],
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.wordpiece_indexer.PretrainedBertIndexer at 0x133381358>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None}

## Prepare vocabulary

We don't need to build the vocab: all that is handled by the token indexer

In [13]:
vocab = Vocabulary()

## Prepare iterator
The iterator is responsible for batching the data and preparing it for input into the model. We'll use the BucketIterator that batches text sequences of smilar lengths together. 

In [14]:
from allennlp.data.iterators import BucketIterator

In [15]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("tokens", "num_tokens")],
                         )

We need to tell the iterator how to numericalize the text data. We do this by passing the vocabulary to the iterator. This step is easy to forget so be careful!

In [16]:
iterator.index_with(vocab)

## Read a sample batch

In [17]:
batch = next(iter(iterator(train_ds)))
batch

{'tokens': {'tokens': tensor([[  101,   100,  2777,  ...,     0,     0,     0],
          [  101,   100,  2302,  ...,   102,     0,     0],
          [  101,   100,  2738,  ...,     0,     0,     0],
          ...,
          [  101,   100,  2424,  ...,  9463, 18075,   102],
          [  101,   100,  3062,  ..., 19728, 29625,   102],
          [  101,   100,   100,  ...,     0,     0,     0]]),
  'tokens-offsets': tensor([[ 1,  2,  3,  ...,  0,  0,  0],
          [ 1,  2,  3,  ..., 15,  0,  0],
          [ 1,  2,  3,  ...,  0,  0,  0],
          ...,
          [ 1,  2,  3,  ..., 15, 16, 17],
          [ 1,  2,  3,  ..., 15, 16, 17],
          [ 1,  2,  3,  ...,  0,  0,  0]]),
  'tokens-type-ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]),
  'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1, 

In [18]:
batch["tokens"]["tokens"]

tensor([[  101,   100,  2777,  ...,     0,     0,     0],
        [  101,   100,  2302,  ...,   102,     0,     0],
        [  101,   100,  2738,  ...,     0,     0,     0],
        ...,
        [  101,   100,  2424,  ...,  9463, 18075,   102],
        [  101,   100,  3062,  ..., 19728, 29625,   102],
        [  101,   100,   100,  ...,     0,     0,     0]])

In [19]:
batch["tokens"]["tokens"].shape


torch.Size([64, 19])

## Prepare Model

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim

In [21]:
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder

class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

## Prepare embeddings

In [22]:
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders.bert_token_embedder import PretrainedBertEmbedder

bert_embedder = PretrainedBertEmbedder(
        pretrained_model="bert-base-uncased",
        top_layer_only=True, # conserve memory
)
word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": bert_embedder},
                                                            # we'll be ignoring masks so we'll need to set this to True
                                                           allow_unmatched_keys = True)

100%|██████████| 407873900/407873900 [01:10<00:00, 5777609.31B/s]


In [23]:
BERT_DIM = word_embeddings.get_output_dim()

class BertSentencePooler(Seq2VecEncoder):
    def forward(self, embs: torch.tensor, 
                mask: torch.tensor=None) -> torch.tensor:
        # extract first token tensor
        return embs[:, 0]
    
    @overrides
    def get_output_dim(self) -> int:
        return BERT_DIM
    
encoder = BertSentencePooler(vocab)

Notice how simple and modular the code for initializing the model is. All the complexity is delegated to each component.

In [24]:
model = BaselineModel(
    word_embeddings, 
    encoder, 
)

In [25]:

if USE_GPU: model.cuda()
else: model

## Basic sanity checks

In [26]:
batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

In [27]:
tokens = batch["tokens"]
labels = batch

In [28]:
tokens

{'tokens': tensor([[  101,   100,  2777,  ...,     0,     0,     0],
         [  101,   100,  2302,  ...,   102,     0,     0],
         [  101,   100,  2738,  ...,     0,     0,     0],
         ...,
         [  101,   100,  2424,  ...,  9463, 18075,   102],
         [  101,   100,  3062,  ..., 19728, 29625,   102],
         [  101,   100,   100,  ...,     0,     0,     0]]),
 'tokens-offsets': tensor([[ 1,  2,  3,  ...,  0,  0,  0],
         [ 1,  2,  3,  ..., 15,  0,  0],
         [ 1,  2,  3,  ...,  0,  0,  0],
         ...,
         [ 1,  2,  3,  ..., 15, 16, 17],
         [ 1,  2,  3,  ..., 15, 16, 17],
         [ 1,  2,  3,  ...,  0,  0,  0]]),
 'tokens-type-ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 0, 0],
         [1, 1, 1

In [29]:
mask = get_text_field_mask(tokens)
mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])

In [30]:
embeddings = model.word_embeddings(tokens)
state = model.encoder(embeddings, mask)
class_logits = model.projection(state)
class_logits

tensor([[ 0.1824, -0.1583],
        [ 0.0538, -0.5639],
        [ 0.3546,  0.0099],
        [ 0.1720,  0.2146],
        [ 0.2468, -0.0498],
        [ 0.1716,  0.0061],
        [ 0.0315, -0.1518],
        [ 0.1567,  0.0910],
        [ 0.2617,  0.0378],
        [ 0.3030,  0.1635],
        [ 0.2562, -0.1694],
        [ 0.0322, -0.0254],
        [ 0.3319, -0.0425],
        [ 0.3415,  0.1019],
        [ 0.5443,  0.0410],
        [ 0.2463, -0.0489],
        [ 0.2101, -0.1463],
        [ 0.5497,  0.0603],
        [ 0.1768,  0.0570],
        [ 0.4695,  0.2415],
        [ 0.3106,  0.3136],
        [ 0.2150, -0.2025],
        [ 0.3638,  0.0986],
        [ 0.0825, -0.0146],
        [ 0.2939, -0.1234],
        [ 0.1673, -0.0813],
        [ 0.1968, -0.1756],
        [ 0.1644,  0.2042],
        [ 0.0044, -0.0088],
        [ 0.2984,  0.0595],
        [ 0.2600, -0.0238],
        [ 0.2343,  0.2755],
        [ 0.1559,  0.1624],
        [ 0.4113,  0.0732],
        [ 0.1674, -0.5165],
        [ 0.3914,  0

In [31]:
model(**batch)

{'class_logits': tensor([[ 0.0879, -0.0800],
         [ 0.1675, -0.1584],
         [ 0.4839, -0.0300],
         [ 0.1695,  0.2763],
         [ 0.0508, -0.2329],
         [ 0.1382, -0.1222],
         [-0.0318, -0.0507],
         [ 0.3662,  0.1041],
         [ 0.2445, -0.2180],
         [ 0.1242, -0.0656],
         [ 0.4090,  0.0185],
         [-0.0245, -0.0287],
         [ 0.2556, -0.2059],
         [ 0.2417, -0.1219],
         [ 0.1659,  0.0105],
         [ 0.3586,  0.0056],
         [ 0.0675,  0.0558],
         [ 0.4029, -0.0824],
         [ 0.4204,  0.1177],
         [ 0.2633,  0.2190],
         [ 0.2416,  0.6000],
         [ 0.2269, -0.1454],
         [ 0.3465, -0.2412],
         [ 0.0892,  0.0506],
         [ 0.5035,  0.0732],
         [-0.0602, -0.4913],
         [ 0.2417,  0.0592],
         [ 0.3030,  0.2401],
         [-0.0476,  0.0693],
         [ 0.3784,  0.2068],
         [ 0.3485, -0.1336],
         [ 0.3404,  0.1547],
         [ 0.0857, -0.0767],
         [ 0.3591, -0.0752]

In [32]:
loss = model(**batch)["loss"]


In [33]:
loss

tensor(0.6968, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [34]:
loss.backward()

In [35]:
[x.grad for x in list(model.encoder.parameters())]

[]

# Train Model

In [36]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [37]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [38]:
metrics = trainer.train()

loss: 0.6930 ||: 100%|██████████| 32/32 [01:16<00:00,  1.94s/it]
loss: 0.6321 ||: 100%|██████████| 32/32 [01:14<00:00,  1.61s/it]


# Generating Predictions

In [39]:
from allennlp.data.iterators import DataIterator
from tqdm import tqdm
from scipy.special import expit # the sigmoid function

def tonp(tsr): return tsr.detach().cpu().numpy()

class Predictor:
    def __init__(self, model: Model, iterator: DataIterator,
                 cuda_device: int=-1) -> None:
        self.model = model
        self.iterator = iterator
        self.cuda_device = cuda_device
        
    def _extract_data(self, batch) -> np.ndarray:
        out_dict = self.model(**batch)
        return expit(tonp(out_dict["class_logits"]))
    
    def predict(self, ds: Iterable[Instance]) -> np.ndarray:
        pred_generator = self.iterator(ds, num_epochs=1, shuffle=False)
        self.model.eval()
        pred_generator_tqdm = tqdm(pred_generator,
                                   total=self.iterator.get_num_batches(ds))
        preds = []
        with torch.no_grad():
            for batch in pred_generator_tqdm:
                batch = nn_util.move_to_device(batch, self.cuda_device)
                preds.append(self._extract_data(batch))
        return np.concatenate(preds, axis=0)

In [40]:
from allennlp.data.iterators import BasicIterator
# iterate over the dataset without changing its order
seq_iterator = BasicIterator(batch_size=64)
seq_iterator.index_with(vocab)

In [41]:
predictor = Predictor(model, seq_iterator, cuda_device=0 if USE_GPU else -1)
train_preds = predictor.predict(train_ds) 
#test_preds = predictor.predict(test_ds)

100%|██████████| 32/32 [02:48<00:00,  3.47s/it]


# A Final Note on Predictors

AllenNLP also provides predictors that take strings as input and outputs model predictions. They're handy if you want to create simple demo or need to make predictions on entirely new data, but since we've already read data as datasets and want to preserve their order, we didn't use them above.

Need to make a reader that will convert the string in the format the that model understands. In this case, similar to the train data. The model is looking for an instance not a string!

In [42]:
import numpy as np
import pandas as pd

In [43]:
fname = 'reddit_thread_label_small_test'

In [44]:
test_ds = reader.read(fname) 
predictor = Predictor(model, seq_iterator, cuda_device=0 if USE_GPU else -1)
test_preds = predictor.predict(test_ds) 
df=pd.read_pickle(fname)
score = np.argmax(test_preds,axis=1)
df['temp'] = df[label_cols].apply(lambda x: np.array(x[label_cols]),axis=1)
df['label'] = df.temp.apply(lambda x:np.argmax(x))
print(f'Score {sum(df.label == score)/len(score)}')

2000it [00:01, 1105.75it/s]
100%|██████████| 32/32 [02:50<00:00,  3.47s/it]


Score 0.88


In [ ]:
# for label in label_cols:
#     score = np.argmax(train_preds,axis=1)
#     argument = np.argmax(np.array(label_cols) == label)
#     score = np.sum(score == argument)/len(train_preds)
    
#     print(f'{label} has a score of {score}')

In [ ]:
# from allennlp.data.fields import TextField, MetadataField, ArrayField

# class NewDatasetReader(DatasetReader):
#     def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
#                  token_indexers: Dict[str, TokenIndexer] = None,
#                  max_seq_len: Optional[int]=config.max_seq_len) -> None:
#         super().__init__(lazy=False)
#         self.tokenizer = tokenizer
#         self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
#         self.max_seq_len = max_seq_len

#     @overrides
#     def text_to_instance(self, tokens: List[Token],
#                          labels: np.ndarray=None,
#                         id: str=None,) -> Instance:
#         sentence_field = TextField(tokens, self.token_indexers)
#         fields = {"tokens": sentence_field}
        
#         fields["label"] = ArrayField(array=labels)
    
#         id_field = MetadataField(np.random.randint(1000))
#         fields["id"] = id_field
#         return Instance(fields)
    
#     @overrides
#     def _read(self, string: str) -> Iterator[Instance]:
#         # Imports data into the tokenizer
#         #for string in strings:
#         print(string)
#         yield self.text_to_instance([Token(x) for x in self.tokenizer(str(string))])
            

In [ ]:
# test_reader = NewDatasetReader(
#     tokenizer=tokenizer,
#     token_indexers={"tokens": token_indexer})

In [ ]:
# test_reader.read("this tutorial was great!")

In [ ]:
# from allennlp.predictors import SentenceTaggerPredictor
# predictor = SentenceTaggerPredictor(model, dataset_reader=reader)
# tag_logits = predictor.predict("The dog ate the apple")['tag_logits']
# tag_ids = np.argmax(tag_logits, axis=-1)
# print([model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])